In [116]:
import numpy as np
import sys
from scipy.optimize import linprog

sys.setrecursionlimit(10000)

31181  
19  
4990 1945  
1142 321  
7390 2945  
10372 4136  
3114 1107  
2744 1022  
3102 1101  
7280 2890  
2624 962  
3020 1060  
2310 805  
2078 689  
3926 1513  
9656 3878  
32708 13504  
4830 1865  
2034 667  
4766 1833  
40006 16553  

Output: 12248

In [92]:
max_volume = int(input())
k_items = int(input())

volumes = []
values = []
wrong_items = 0

for i in range(k_items):
    volume, value = list(map(int, input().split()))
    
    if volume <= max_volume:
        volumes.append(volume)
        values.append(value)
    else:
        wrong_items += 1
        
k_items -= wrong_items
volumes = np.array(volumes)
values = np.array(values)

31181
19
4990 1945 
1142 321  
7390 2945
10372 4136
3114 1107 
2744 1022 
3102 1101 
7280 2890  
2624 962  
3020 1060 
2310 805  
2078 689
3926 1513 
9656 3878
32708 13504 
4830 1865
2034 667
4766 1833
40006 16553


In [249]:
def knapsack_linprog(volumes, values, max_volume):
    lp_res = linprog(-values, 
                     A_ub=volumes[None, :], 
                     b_ub=np.array(max_volume),
                     bounds=(0, 1))

    return np.floor(-lp_res.fun)

# Жадность по удельной стоимости объектов
def knapsack_greedy_1(volumes, values, max_volume, is_float=False, is_sorted=False):
    if is_sorted:
        sorted_unit_cost_inds = np.arange(values.size)
    else:
        sorted_unit_cost_inds = np.argsort(values / volumes)[::-1]
    
    residual_volume = max_volume
    greedy_value = 0
    
    for i in sorted_unit_cost_inds:
        if residual_volume > volumes[i]:
            greedy_value += values[i]
            residual_volume -= volumes[i]
        else:
            if is_float:
                greedy_value += values[i] * residual_volume / volumes[i]
                break
    
    return greedy_value

# Жадность по ценности объектов
def knapsack_greedy_2(volumes, values, max_volume):
    sorted_values_inds = np.argsort(values)[::-1]
    
    residual_volume = max_volume
    greedy_value = 0
    
    for i in sorted_values_inds:
        if residual_volume > volumes[i]:
            greedy_value += values[i]
            residual_volume -= volumes[i]
    
    return greedy_value

def knapsack_greedy(volumes, values, max_volume):
    first_greedy_cost = knapsack_greedy_1(volumes, values, max_volume)
    second_greedy_cost = knapsack_greedy_2(volumes, values, max_volume)

    return max(first_greedy_cost, second_greedy_cost)

def knapsack_lbound(volumes, values, max_volume):
    return knapsack_greedy(volumes, values, max_volume)

def knapsack_ubound(volumes, values, max_volume):
    return knapsack_greedy_1(volumes, values, max_volume, True, True)

In [250]:
def complete_bust(ind, volumes, values, curr_sum, 
                  max_volume, lower_bound=0):
    if max_volume < 0:
        return 0
    
    if ind == volumes.size or max_volume < volumes.min():
        return curr_sum
    
    left_res, right_res = 0, 0
    
    left_upper_bound = curr_sum
    right_upper_bound = curr_sum + values[ind]
    
    if ind + 1 < values.size:
        left_upper_bound += knapsack_ubound(volumes[ind+1:], 
                                            values[ind+1:], 
                                            max_volume)
        right_upper_bound += knapsack_ubound(volumes[ind+1:], 
                                             values[ind+1:], 
                                             max_volume-volumes[ind])
    
    if left_upper_bound > lower_bound:
        left_res = complete_bust(ind+1, volumes, values, curr_sum, max_volume, lower_bound)

    if right_upper_bound > lower_bound:
        curr_sum += values[ind]
        max_volume -= volumes[ind]
        
        lower_bound = max(lower_bound, left_res)
        
        right_res = complete_bust(ind+1, volumes, values, curr_sum, max_volume, lower_bound)

    return max(left_res, right_res, lower_bound)

In [279]:
sys.setrecursionlimit(1000)

RecursionError: cannot set the recursion limit to 10 at the recursion depth 28: the limit is too low

In [278]:
%%time
lower_bound = knapsack_lbound(volumes, values, max_volume)
inds = np.argsort(values / volumes)[::-1]
tmp_volumes = volumes[inds]
tmp_values = values[inds]

print(complete_bust(0, tmp_volumes, tmp_values, 0, max_volume, lower_bound))

12248
CPU times: user 14.9 ms, sys: 3.97 ms, total: 18.9 ms
Wall time: 15 ms
